#**Hospital Patient Data Analytics**

##**Task-1**

Loading the patient dataset and showing summary.

In [ ]:
import pandas as pd
patients = pd.read_csv('Patient_Data.csv')
billing = pd.read_csv('Billing_Data.csv')

print("Patients information: ")
patients.info()
print("Billing information: ")
billing.info()

Patients information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
Billing information: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [ ]:
print("Patients firts 5 rows: ")
patients.head()


Patients firts 5 rows: 


,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [ ]:
print("Billing firts 5 rows: ")
billing.head()

Billing firts 5 rows: 


,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


##**Task-2**

Selecting only columns relevant for billing: ['Patient ID','Department','Doctor','Billamount']

In [ ]:
relevant_cols = ['Patient ID','Department','Doctor','Billamount']
existing_relevant = [c for c in relevant_cols if c in patients.columns]
print(f"Relevant columns in Patient file: {existing_relevant}")

patient_billing_view = patients[[c for c in existing_relevant if c in patients.columns]].copy()
print("\nPatient billing view (from paient file) columns:", patient_billing_view.columns.tolist())

Relevant columns in Patient file: ['Department', 'Doctor']

Patient billing view (from paient file) columns: ['Department', 'Doctor']


##**Task-3**

Dropping administrative columns like ['ReceptionistD','CheckinTime']

In [ ]:
admin_cols_to_drop = ['ReceptionistD','CheckinTime']
cols_to_drop = [c for c in admin_cols_to_drop if c in patients.columns]
if cols_to_drop :
  print(f"Dropping columns: {cols_to_drop}")
  patients = patients.drop(columns=cols_to_drop,inplace=True)
else :
  print("No administrative columns to drop")

No administrative columns to drop


##**Task-4**

Finding total bill amount per department by using groupby.

In [ ]:
if 'Billamount' in billing.columns:
    bill_source = billing
    print("\nUsing Billing file for BillAmount aggregation.")
elif 'Billamount' in patients.columns:
    bill_source = patients
    print("\nUsing Patient file for BillAmount aggregation.")
else:
    bill_source = None
    print("\nNo Billamount column found in either file, aggregation cannot be computed directly")

if bill_source is not None:
    if 'Department' in bill_source.columns and 'BillAmount' in bill_source.columns:
        dept_total = bill_source.groupbt('Departmeent',dropna=False)['BillAmount'].sum().reset_index().rename(columns={'BillAmount':'TotalBillAmount'})
        print("\nTotal bill amount per department:")
        print(dept_total.head(20).to_string(index=False))
    else :
      print("\nDepartment or BillAmount missing from billing source; attempting to mergefirst before grouping.")
else :
  print("\nPatientID not found in patient file; cannot duplicate.")



No Billamount column found in either file, aggregation cannot be computed directly

PatientID not found in patient file; cannot duplicate.


##**Task-5**

Removing **duplicate** patient records based on PatientID

In [ ]:
if 'PatientID' in patients.columns:
    before_dups = patients.shape[0]
    patients = patients.drop_duplicates(subset='PatientID',keep='first').reset_index(drop=True)
    after_dups = patients.shape[0]
    print(f"\nRemoved duplicates in patients based on PatientID: {before_dups - after_dups} duplicates dropped.")
else :
  print("\nPatientID not found in patient file; cannot duplicate.")



Removed duplicates in patients based on PatientID: 1 duplicates dropped.


##**Task-6**

Filling missing BillAmount values with the **mean** bill amount

In [ ]:
if 'BillAmount' in billing.columns:
    mean_bill = billing['BillAmount'].mean()
    na_before = billing['BillAmount'].isna().sum()
    billing['BillAmount'] = billing['BillAmount'].fillna(mean_bill)
    na_after = billing['BillAmount'].isna().sum()
    print(f"\nFilled missing Billing BillAmount values with mean ({mean_bill:.2f}). NaNs before: {na_before}, after: {na_after}")
else:
    # As fallback, if patients has BillAmount, fill there
    if 'BillAmount' in patients.columns:
        mean_bill = patients['BillAmount'].mean()
        na_before = patients['BillAmount'].isna().sum()
        patients['BillAmount'] = patients['BillAmount'].fillna(mean_bill)
        na_after = patients['BillAmount'].isna().sum()
        print(f"\nFilled missing Patient file BillAmount values with mean ({mean_bill:.2f}). NaNs before: {na_before}, after: {na_after}")
    else:
        print("\nNo BillAmount column found in either file to fill missing values.")


Filled missing Patient file BillAmount values with mean (6233.33). NaNs before: 2, after: 0


##**Task-7**

Merging the billing dataset with patient dataset on PatientID.

In [ ]:
if 'PatientID' in patients.columns and 'PatientID' in billing.columns:
    merged = pd.merge(patients, billing, on='PatientID', how='left', suffixes=('_patient', '_billing'))
    print(f"\nMerged patients and billing. Result shape: {merged.shape}")
else:
    print("\nPatientID missing in one of the files; attempting index-wise concat as fallback.")
    merged = pd.concat([patients.reset_index(drop=True), billing.reset_index(drop=True)], axis=1)
    print(f"Concatenated datasets side-by-side. Result shape: {merged.shape}")

print("\nMerged dataset preview:")
print(merged.head())


Merged patients and billing. Result shape: (5, 9)

Merged dataset preview:
   PatientID     Name   Department     Doctor   BillAmount  ReceptionistID  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000               1   
1        102      Bob    Neurology   Dr. John  6233.333333               2   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000               1   
3        104    David   Cardiology  Dr. Smith  6200.000000               3   
4        105      Eva  Dermatology   Dr. Rose  6233.333333               2   

        CheckInTime  InsuranceCovered  FinalAmount  
0  2023-01-10 09:00              2000         3000  
1  2023-01-11 10:30              1500         3500  
2  2023-01-12 11:00              2500         5000  
3  2023-01-13 12:00              3000         3200  
4  2023-01-14 08:45              1000         4000  


##**Task-8**

Concatenating an additional DataFrame that contains new patients for the current week (row-wise).

In [ ]:
import numpy as np
new_patients = pd.DataFrame([
    {'PatientID': 'P1001', 'Name': 'Test Patient A', 'Department': 'Cardiology', 'Doctor': 'Dr. Heart', 'BillAmount': np.nan},
    {'PatientID': 'P1002', 'Name': 'Test Patient B', 'Department': 'Orthopedics', 'Doctor': 'Dr. Bone', 'BillAmount': 2500.0},
])
print("\nNew patients DataFrame (example):")
print(new_patients)

patients_updated = pd.concat([patients, new_patients], ignore_index=True, sort=False)
print(f"\nAfter row-wise concatenation, patients_updated shape: {patients_updated.shape}")


New patients DataFrame (example):
  PatientID            Name   Department     Doctor  BillAmount
0     P1001  Test Patient A   Cardiology  Dr. Heart         NaN
1     P1002  Test Patient B  Orthopedics   Dr. Bone      2500.0

After row-wise concatenation, patients_updated shape: (7, 7)


##**Task-9**

Concatenating new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [ ]:
extra_billing_cols = pd.DataFrame({
    'InsuranceCovered': [True, False] + [np.nan] * (patients_updated.shape[0] - 2),
    'FinalAmount': [0.0, 2500.0] + [np.nan] * (patients_updated.shape[0] - 2),
})

extra_billing_cols = extra_billing_cols.reindex(range(patients_updated.shape[0])).reset_index(drop=True)

patients_with_extra_cols = pd.concat([patients_updated.reset_index(drop=True), extra_billing_cols], axis=1)
print("\nPatients with extra billing columns preview:")
print(patients_with_extra_cols.head())


Patients with extra billing columns preview:
  PatientID     Name   Department     Doctor   BillAmount  ReceptionistID  \
0       101    Alice   Cardiology  Dr. Smith  5000.000000             1.0   
1       102      Bob    Neurology   Dr. John  6233.333333             2.0   
2       103  Charlie  Orthopedics    Dr. Lee  7500.000000             1.0   
3       104    David   Cardiology  Dr. Smith  6200.000000             3.0   
4       105      Eva  Dermatology   Dr. Rose  6233.333333             2.0   

        CheckInTime InsuranceCovered  FinalAmount  
0  2023-01-10 09:00             True          0.0  
1  2023-01-11 10:30            False       2500.0  
2  2023-01-12 11:00              NaN          NaN  
3  2023-01-13 12:00              NaN          NaN  
4  2023-01-14 08:45              NaN          NaN  


#**Filling**

Final cleaned Dataset with accurate info

In [ ]:
if 'BillAmount' in patients_with_extra_cols.columns:
    overall_mean_bill = patients_with_extra_cols['BillAmount'].mean(skipna=True)
    patients_with_extra_cols['BillAmount'] = patients_with_extra_cols['BillAmount'].fillna(overall_mean_bill)
    print(f"\nFilled remaining missing BillAmount in patients_with_extra_cols with mean ({overall_mean_bill:.2f}).")



Filled remaining missing BillAmount in patients_with_extra_cols with mean (5611.11).


All missing values handled, merged dataset across PatientID.

In [ ]:
if 'PatientID' in patients_with_extra_cols.columns:
    before = patients_with_extra_cols.shape[0]
    patients_with_extra_cols = patients_with_extra_cols.drop_duplicates(subset=['PatientID'], keep='first').reset_index(drop=True)
    after = patients_with_extra_cols.shape[0]
    print(f"Dropped {before - after} duplicate patient rows in final dataset (if any).")

Dropped 0 duplicate patient rows in final dataset (if any).


Ability to perform further analytics on department-wise revenue or doctor performance.

In [ ]:
if 'Department' in patients_with_extra_cols.columns and 'BillAmount' in patients_with_extra_cols.columns:
    final_dept_revenue = patients_with_extra_cols.groupby('Department', dropna=False)['BillAmount'].sum().reset_index().rename(columns={'BillAmount':'TotalBillAmount'})
    print("\nFinal Department-wise total billing (from final dataset):")
    print(final_dept_revenue.to_string(index=False))
else:
    print("\nCannot compute final department revenue: Department or BillAmount missing.")


Final Department-wise total billing (from final dataset):
 Department  TotalBillAmount
 Cardiology     16811.111111
Dermatology      6233.333333
  Neurology      6233.333333
Orthopedics     10000.000000


In [ ]:
print("\nProcessing complete. The cleaned dataset is ready for further analytics (department-wise revenue, doctor performance, etc.).")


Processing complete. The cleaned dataset is ready for further analytics (department-wise revenue, doctor performance, etc.).
